# EDA pour identifer la structure des donnees Open-Source de MTL

In [1]:
!pip install geopandas

     |████████████████████████████████| 972kB 2.8MB/s 
     |████████████████████████████████| 14.7MB 298kB/s 
     |████████████████████████████████| 10.9MB 45.3MB/s 


In [2]:
import pandas as pd
import io
import os
from google.colab import drive
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np

### 1) Connection to the drive

In [3]:
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
# Read data from Open data Montreal
spatial_boroughs = gpd.read_file('/content/drive/My Drive/Data/YCBS-299/quartiers-socio.geojson', encoding='utf-8')
spatial_casernes = gpd.read_file('/content/drive/My Drive/Data/YCBS-299/territoires-administratifs-casernes.geojson', encoding='utf-8')

### 2) Create a table with all days, shifts and stations

In [5]:
# Load datasets
interventions_15 = pd.read_csv('/content/drive/My Drive/Data/YCBS-299/donneesouvertes-interventions-sim.csv')
list_casernes = interventions_15['CASERNE'].unique()

In [6]:
def create_date_table(start='2005-01-01', end='2020-07-10'):
  df = pd.DataFrame({"Date": pd.date_range(start, end)})
  return df 

In [ ]:
# Create date table
shifts = [1,2,3]

appended_data = []
for i in list_casernes:
  for j in shifts:
    dates = create_date_table(start='2005-01-01', end='2020-06-30')
    dates['Date'] = dates['Date'].dt.strftime('%m/%d/%Y')
    dates['Station'] = i
    dates['Shift'] = j
    appended_data.append(dates)

# see pd.concat documentation for more info
dates = pd.concat(appended_data)

In [ ]:
dates.head()

,Date,Station,Shift
0,01/01/2005,77,1
1,01/02/2005,77,1
2,01/03/2005,77,1
3,01/04/2005,77,1
4,01/05/2005,77,1


In [ ]:
# Print to csv
dates.to_csv('/content/drive/My Drive/Data/YCBS-299/time_space_frame.csv', index=False)

### 3) Table for Social Housing Data

In [22]:
social_housing =  pd.read_csv('/content/drive/My Drive/Data/YCBS-299/logsoc_donneesouvertes_20191231.csv', encoding='utf-8')

In [31]:
social_housing = social_housing[['nom_projet','an_orig' ,'nb_log', 'projettype', 'longitude', 'latitude']]
social_housing.head()

,nom_projet,an_orig,nb_log,projettype,longitude,latitude
0,Saint-Sulpice,1971.0,150,HLM,-73.645933,45.556394
1,Adrien Trudeau,1994.0,32,HLM,-73.672437,45.445957
2,Montmorency,1978.0,80,HLM,-73.560734,45.486068
3,Place Normandie,1972.0,168,HLM,-73.623530,45.623784
4,Étienne-Desmarteaux,1978.0,117,HLM,-73.580121,45.558369


Faire une table annees 2005 a 2020

In [41]:
appended_data = []
for years in range(2005,2021):
    year = pd.DataFrame(np.array([years]),columns=['Year'])
    appended_data.append(year)

# see pd.concat documentation for more info
year_frame = pd.concat(appended_data)

# see pd.concat documentation for more info
year_frame = pd.concat(appended_data)


,Year
0,2016
0,2017
0,2018
0,2019
0,2020


In [44]:
import sqlite3
qry = '''
    select  
        year_frame.Year Year,
        social_housing.nom_projet nom_projet,
        social_housing.an_orig an_orig,
        social_housing.nb_log nb_log,
        social_housing.projettype projettype,
        social_housing.longitude longitude,
        social_housing.latitude latitude
    from
        year_frame left join social_housing on
        social_housing.an_orig <= year_frame.Year
    '''

#Make the db in memory
conn = sqlite3.connect(':memory:')

#write the tables
year_frame.to_sql('year_frame', conn, index=False)
social_housing.to_sql('social_housing', conn, index=False)
social_housing_yr = pd.read_sql_query(qry, conn)
social_housing_yr.head()

,Year,nom_projet,an_orig,nb_log,projettype,longitude,latitude
0,2005,Saint-Sulpice,1971.0,150,HLM,-73.645933,45.556394
1,2005,Adrien Trudeau,1994.0,32,HLM,-73.672437,45.445957
2,2005,Montmorency,1978.0,80,HLM,-73.560734,45.486068
3,2005,Place Normandie,1972.0,168,HLM,-73.623530,45.623784
4,2005,Étienne-Desmarteaux,1978.0,117,HLM,-73.580121,45.558369


In [48]:
list_casernes = [None] * len(social_housing_yr)
list_boroughs = [None] * len(social_housing_yr)

for house in range(0, len(social_housing_yr)-1):

  longitude = social_housing_yr['longitude'].iloc[house]
  latitude  = social_housing_yr['latitude'].iloc[house]
  p = Point(longitude, latitude)

  for caserne in range(0, len(spatial_casernes)-1):
    if p.within(spatial_casernes['geometry'].iloc[caserne]):
      list_casernes[house] = spatial_casernes['NOM_CAS_AD'].iloc[caserne]

  for borough in range(0, len(spatial_boroughs)-1):
    if p.within(spatial_boroughs['geometry'].iloc[borough]):
      list_boroughs[house] = spatial_boroughs['Arrondissement'].iloc[borough]

In [49]:
key_caserne  = pd.DataFrame(list_casernes, columns = ["Station"])
key_boroughs = pd.DataFrame(list_boroughs, columns = ["Borough"])

keys = key_caserne.join(key_boroughs)
keys.head()

,Station,Borough
0,Caserne 49,Ahuntsic-Cartierville
1,Caserne 64,Lachine
2,Caserne 15,Le Sud-Ouest
3,Caserne 18,Montréal-Nord
4,Caserne 50,Rosemont–La Petite-Patrie


In [50]:
social_housing_keys = social_housing_yr.join(keys)

In [61]:
social_housing_keys.head()

,Year,nom_projet,an_orig,nb_log,projettype,longitude,latitude,Station,Borough
0,2005,Saint-Sulpice,1971.0,150,HLM,-73.645933,45.556394,Caserne 49,Ahuntsic-Cartierville
1,2005,Adrien Trudeau,1994.0,32,HLM,-73.672437,45.445957,Caserne 64,Lachine
2,2005,Montmorency,1978.0,80,HLM,-73.560734,45.486068,Caserne 15,Le Sud-Ouest
3,2005,Place Normandie,1972.0,168,HLM,-73.623530,45.623784,Caserne 18,Montréal-Nord
4,2005,Étienne-Desmarteaux,1978.0,117,HLM,-73.580121,45.558369,Caserne 50,Rosemont–La Petite-Patrie


In [58]:
social_housing_final = social_housing_keys.pivot_table(index=['Year', 'Station'], columns='projettype', values='nb_log', aggfunc=np.sum)

In [59]:
social_housing_final.reset_index(level=0, inplace=True)
social_housing_final.reset_index(level=0, inplace=True)
social_housing_final['Station_'] = social_housing_final['Station'].str.slice(start=-2).astype(int)
social_housing_final = social_housing_final.drop(['Station'], axis=1)
social_housing_final.columns = ['Year','COOP_sum','HLM_sum', 'OBNL_sum', 'OMHM_sum','SHDM_sum', 'Station']
social_housing_final=social_housing_final.fillna(0)
social_housing_final.head()

,Year,COOP_sum,HLM_sum,OBNL_sum,OMHM_sum,SHDM_sum,Station
0,2005,215.0,876.0,48.0,26.0,29.0,3
1,2005,183.0,454.0,784.0,0.0,384.0,4
2,2005,133.0,1112.0,186.0,0.0,64.0,5
3,2005,113.0,20.0,92.0,0.0,0.0,8
4,2005,147.0,881.0,126.0,0.0,135.0,9


In [60]:
social_housing_final.to_csv('/content/drive/My Drive/Data/YCBS-299/social_housing.csv', index=False)